In [1]:
import pandas as pd
import datetime as dt
import time
import requests
import nltk

In [2]:
def query_pushshift(subreddit, kind = 'submission', day_window = 30, n = 5):
    SUBFIELDS = ['title', 'selftext', 'subreddit', 'created_utc', 'author', 'num_comments', 'score', 'is_self']
    
    # establish base url and stem
    BASE_URL = f"https://api.pushshift.io/reddit/search/{kind}" # also known as the "API endpoint" 
    stem = f"{BASE_URL}?subreddit={subreddit}&size=500" # always pulling max of 500
    
    # instantiate empty list for temp storage
    posts = []
    
    # implement for loop with `time.sleep(2)`
    for i in range(1, n + 1):
        URL = "{}&after={}d".format(stem, day_window * i)
        print("Querying from: " + URL)
        response = requests.get(URL)
        assert response.status_code == 200
        mine = response.json()['data']
        df = pd.DataFrame.from_dict(mine)
        posts.append(df)
        time.sleep(2)
    
    # pd.concat storage list
    full = pd.concat(posts, sort=False)
    
    # if submission
    if kind == "submission":
        # select desired columns
        full = full[SUBFIELDS]
        # drop duplicates
        full.drop_duplicates(inplace = True)
        # select `is_self` == True
        full = full.loc[full['is_self'] == True]

    # create `timestamp` column
    full['timestamp'] = full["created_utc"].map(dt.date.fromtimestamp)
    
    print("Query Complete!")    
    return full 

In [3]:
subreddit1 = 'technology'
subreddit_1 = query_pushshift(subreddit1)
subreddit2 = 'flying'
subreddit_2 = query_pushshift(subreddit2)

Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=technology&size=500&after=30d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=technology&size=500&after=60d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=technology&size=500&after=90d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=technology&size=500&after=120d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=technology&size=500&after=150d
Query Complete!
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=flying&size=500&after=30d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=flying&size=500&after=60d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=flying&size=500&after=90d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=flying&size=500&after=120d
Querying from: https://api.pushshift.io/reddit/searc

In [5]:
#combine = subreddit_1 + subreddit_2
subreddit_1.to_csv('datasets/subreddit1.csv')
subreddit_2.to_csv('datasets/subreddit2.csv')
combined = pd.concat([subreddit_1, subreddit_2])
combined.to_csv('datasets/combined.csv')